In [4]:
# Restric calcuation of psl accuracy to only those cell lines/Gene pair that has information
# contribution to the final output

### Only active
# predict everything
# predict with data

### Only essential
# predict everything
# predict with data

### both rules
# predict everything
# predict with at least one type of data
# predict with both types of data

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import compare_y

%matplotlib inline


In [13]:
infer_file = "../psl/result/first_model_cross_val_fold{0}_result{1}.txt"
train_file = "../psl/data/first_model/seed0/cross_val/5fold/fold{0}_train.txt"
test_file = "../psl/data/first_model/seed0/cross_val/5fold/fold{0}_val.txt"

In [40]:
# Define some helper functions 
def get_gene_drug_dict():
    drug_target_file = "../psl/data/first_model/drug_target.txt"
    drug_target_df = pd.read_csv(drug_target_file, delimiter="\t", header=None)
    drug_target_dict = {}
    for index, row in drug_target_df.iterrows():
        if row[1] in drug_target_dict.keys():
            drug_target_dict[row[1]].append(row[0])
        else:
            drug_target_dict[row[1]] = [row[0]]
    return drug_target_dict


def get_active_cell_drug_set(gene_drug_dict):
    active_df = pd.read_csv("../psl/data/first_model/active.txt", delimiter="\t", header=None)
    active_cell_drug_pairs = []
    for index, row in active_df.iterrows():
        if row[1] in gene_drug_dict.keys():
            for drug in gene_drug_dict[row[1]]:
                active_cell_drug_pairs.append(row[0] + drug)
    return set(active_cell_drug_pairs)


def get_essential_cell_drug_set(gene_drug_dict):
    essential_df = pd.read_csv("../psl/data/first_model/essential.txt", delimiter="\t", header=None)
    cell_drug_pairs = []
    for index, row in essential_df.iterrows():
        if row[1] in gene_drug_dict.keys():
            for drug in gene_drug_dict[row[1]]:
                cell_drug_pairs.append(row[0] + drug)
    return set(cell_drug_pairs)
    
def get_sensitive_cell_drug_set():
    sensitive_df = pd.read_csv("../psl/data/first_model/sensitive_truth.txt", delimiter="\t", header=None)
    cell_drug_pairs = []
    for index, row in sensitive_df.iterrows():
        cell_drug_pairs.append(row[0] + row[1])
    return set(cell_drug_pairs)

gene_drug_dict = get_gene_drug_dict()
active_set = get_active_cell_drug_set(gene_drug_dict)
essential_set = get_essential_cell_drug_set(gene_drug_dict)

In [47]:
# helper function to get overlap between a set and a df
def get_df_overlap(df, cell_drug_set):
    new_rows = []
    for index, row in df.iterrows():
        if row.cell_drug_pair in cell_drug_set:
            new_rows.append(dict(row))
    return pd.DataFrame(new_rows)

In [51]:
# Only active rule
rows = []
for fold in range(1, 6):
    infer_df = compare_y.load_data(infer_file.format(fold, "_activeRule"))
    tr_df = compare_y.load_data(train_file.format(fold))
    val_df = compare_y.load_data(test_file.format(fold))
    tr_mse, _, _ = compare_y.calculate_accuracy(tr_df, infer_df)
    val_mse, _, _ = compare_y.calculate_accuracy(val_df, infer_df)
    
    tr_df_has_data = get_df_overlap(tr_df, active_set)
    val_df_has_data = get_df_overlap(val_df, active_set)
    tr_mse_has_data, _, _ = compare_y.calculate_accuracy(tr_df_has_data, infer_df)
    val_mse_has_data, _, _ = compare_y.calculate_accuracy(val_df_has_data, infer_df)
    
    rows.append({"train_all_data": tr_mse, "test_all_data": val_mse, 
                 "train_known_active": tr_mse_has_data, "test_known_active": val_mse_has_data})
df = pd.DataFrame(rows)
print df

   test_all_data  test_known_active  train_all_data  train_known_active
0       0.084362           0.078129        0.083778            0.077950
1       0.083556           0.079141        0.083995            0.077713
2       0.086090           0.077252        0.083358            0.078181
3       0.083175           0.078342        0.084101            0.077924
4       0.082342           0.077115        0.084274            0.078194


In [53]:
# Only essential rule
rows = []
for fold in range(1, 6):
    infer_df = compare_y.load_data(infer_file.format(fold, "_essentialRule"))
    tr_df = compare_y.load_data(train_file.format(fold))
    val_df = compare_y.load_data(test_file.format(fold))
    tr_mse, _, _ = compare_y.calculate_accuracy(tr_df, infer_df)
    val_mse, _, _ = compare_y.calculate_accuracy(val_df, infer_df)
    
    tr_df_has_data = get_df_overlap(tr_df, essential_set)
    val_df_has_data = get_df_overlap(val_df, essential_set)
    
    tr_mse_has_data, _, _ = compare_y.calculate_accuracy(tr_df_has_data, infer_df)
    val_mse_has_data, _, _ = compare_y.calculate_accuracy(val_df_has_data, infer_df)
    
    rows.append({"train_all_data": tr_mse, "test_all_data": val_mse, 
                 "train_known_essential": tr_mse_has_data, "test_known_essential": val_mse_has_data})
df = pd.DataFrame(rows)
print df

   test_all_data  test_known_essential  train_all_data  train_known_essential
0       0.295679              0.126622        0.292916               0.123591
1       0.303576              0.134389        0.290933               0.121748
2       0.291130              0.117132        0.294033               0.125809
3       0.298901              0.123846        0.292077               0.124133
4       0.278051              0.119317        0.297381               0.125726


In [58]:
# both rules
rows = []
for fold in range(1, 6):
    infer_df = compare_y.load_data(infer_file.format(fold, ""))
    tr_df = compare_y.load_data(train_file.format(fold))
    val_df = compare_y.load_data(test_file.format(fold))
    tr_mse, _, _ = compare_y.calculate_accuracy(tr_df, infer_df)
    val_mse, _, _ = compare_y.calculate_accuracy(val_df, infer_df)
    
    tr_df_has_data = get_df_overlap(tr_df, essential_set.union(active_set))
    val_df_has_data = get_df_overlap(val_df, essential_set.union(active_set))
    
    tr_mse_has_data, _, _ = compare_y.calculate_accuracy(tr_df_has_data, infer_df)
    val_mse_has_data, _, _ = compare_y.calculate_accuracy(val_df_has_data, infer_df) 

    tr_df_both_data = get_df_overlap(tr_df, essential_set.intersection(active_set))
    val_df_both_data = get_df_overlap(val_df, essential_set.intersection(active_set))
    
    
    tr_mse_both_data, _, _ = compare_y.calculate_accuracy(tr_df_both_data, infer_df)
    val_mse_both_data, _, _ = compare_y.calculate_accuracy(val_df_both_data, infer_df)

    
    rows.append({"train_all": tr_mse, "test_all": val_mse, 
                 "train_has_>=1_type_of_data": tr_mse_has_data, "test_has_>=1_type_of_data": val_mse_has_data, 
                 "train_both_data": tr_mse_both_data, "test_both_data": val_mse_both_data})

    df = pd.DataFrame(rows)
print df

   test_all  test_both_data  test_has_>=1_type_of_data  train_all  \
0  0.090805        0.103963                   0.084923   0.090158   
1  0.091951        0.107690                   0.087659   0.089893   
2  0.091658        0.093095                   0.082895   0.089947   
3  0.088727        0.099766                   0.084892   0.090700   
4  0.088351        0.100232                   0.083458   0.090779   

   train_both_data  train_has_>=1_type_of_data  
0         0.100168                    0.084713  
1         0.099321                    0.084052  
2         0.102757                    0.085222  
3         0.101147                    0.084744  
4         0.101219                    0.085087  
